# Segmenting remote sensing imagery with point prompts

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/sam2_point_prompts.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/sam2_point_prompts.ipynb)

This notebook shows how to generate object masks from point prompts with the Segment Anything Model 2 (SAM 2).

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator.

## Install dependencies

Uncomment and run the following cell to install the required dependencies.

In [9]:
 #%pip install -U segment-geospatial

## Import libraries

In [60]:
import leafmap
from samgeo import SamGeo2, regularize

## Create an interactive map

In [61]:
m = leafmap.Map(center=[47.653287, -117.588070], zoom=16, height="800px")
m.add_basemap("Satellite")
m

Map(center=[47.653287, -117.58807], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## Download a sample image

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map. If no geometry is drawn, the default bounding box will be used.

In [3]:
m.user_roi_bounds()

[-117.5982, 47.6536, -117.5955, 47.6549]

In [4]:
if m.user_roi is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-117.6029, 47.65, -117.5936, 47.6563]

In [5]:
image = "satellite.tif"
leafmap.map_tiles_to_geotiff(
    output=image, bbox=bbox, zoom=18, source="Satellite", overwrite=True
)

Downloaded image 1/6
Downloaded image 2/6
Downloaded image 3/6
Downloaded image 4/6
Downloaded image 5/6
Downloaded image 6/6
Saving GeoTIFF. Please wait...
Image saved to satellite.tif


You can also use your own image. Uncomment and run the following cell to use your own image.

In [6]:
# image = '/path/to/your/own/image.tif'

Display the downloaded image on the map.

In [7]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

Map(bottom=23424467.0, center=[47.65425, -117.59685], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [9]:
m.user_rois

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-117.598182, 47.653565],
      [-117.598182, 47.654917],
      [-117.595452, 47.654917],
      [-117.595452, 47.653565],
      [-117.598182, 47.653565]]]}},
  {'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'Point', 'coordinates': [-117.59756, 47.654189]}},
  {'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'Point', 'coordinates': [-117.596482, 47.653988]}},
  {'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'Point', 'coordinates': [-117.59678, 47.654355]}}]}

## Initialize SAM class

Set `automatic=False` to enable the `SAM2ImagePredictor`.

In [10]:
sam = SamGeo2(
    model_id="sam2-hiera-large",
    automatic=False,
)

Specify the image to segment.

In [11]:
sam.set_image(image)

## Segment the image

Use the `predict_by_points()` method to segment the image with specified point coordinates. You can use the draw tools to add place markers on the map. If no point is added, the default sample points will be used.


In [12]:
if m.user_rois is not None:
    point_coords_batch = m.user_rois
else:
    point_coords_batch = [
        [-117.599896, 47.655345],
        [-117.59992, 47.655167],
        [-117.599928, 47.654974],
        [-117.599518, 47.655337],
    ]

In [13]:
print(m.user_rois)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-117.598182, 47.653565], [-117.598182, 47.654917], [-117.595452, 47.654917], [-117.595452, 47.653565], [-117.598182, 47.653565]]]}}, {'type': 'Feature', 'properties': {}, 'geometry': {'type': 'Point', 'coordinates': [-117.59756, 47.654189]}}, {'type': 'Feature', 'properties': {}, 'geometry': {'type': 'Point', 'coordinates': [-117.596482, 47.653988]}}, {'type': 'Feature', 'properties': {}, 'geometry': {'type': 'Point', 'coordinates': [-117.59678, 47.654355]}}]}


Segment the objects using the point prompts and save the output masks.

In [15]:
import geopandas as gpd
from shapely.geometry import shape

# Convert the FeatureCollection to a GeoDataFrame
features = m.user_rois["features"]
geoms = [shape(f["geometry"]) for f in features]
gdf = gpd.GeoDataFrame(geometry=geoms, crs="EPSG:4326")

# Keep only points
points = gdf[gdf.geometry.type == "Point"]

# Convert to list of [x, y] coordinates
point_coords_batch = [[pt.x, pt.y] for pt in points.geometry]

# Run prediction
sam.predict_by_points(
    point_coords_batch=point_coords_batch,
    point_crs="EPSG:4326",
    output="mask.tif",
    dtype="uint8",
)


## Display the result

Add the segmented image to the map.

In [16]:
m.add_raster("mask.tif", cmap="viridis", nodata=0, opacity=0.7, layer_name="Mask")
m

Map(bottom=46848498.0, center=[47.65425, -117.59685], controls=(ZoomControl(options=['position', 'zoom_in_text…

![image](https://github.com/user-attachments/assets/49e413b9-e159-4d72-bf23-a0318bc82d44)

## Use an existing vector dataset as points prompts

Alternatively, you can specify a file path or HTTP URL to a vector dataset containing point geometries.

In [62]:
geojson = "https://github.com/opengeos/datasets/releases/download/places/wa_building_centroids.geojson"

Display the vector dataawr on the map.

In [63]:
m = leafmap.Map()
m.add_raster(image, layer_name="Image")
m.add_circle_markers_from_xy(
    geojson, radius=3, color="red", fill_color="yellow", fill_opacity=0.8
)
m

Map(center=[47.65425, -117.59685], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

![image](https://github.com/user-attachments/assets/f0d3ff1e-15fa-4bd3-ac15-637e8d63527d)

## Segment image with a vector dataset

Segment the image using the specified file path to the vector dataset.

In [42]:
import requests

url = "https://github.com/opengeos/datasets/releases/download/places/wa_building_centroids.geojson"
response = requests.get(url)
geojson = response.json()  # Now geojson is a Python dictionary


In [54]:
from shapely.geometry import shape
import requests

# Load GeoJSON from GitHub
url = "https://github.com/opengeos/datasets/releases/download/places/wa_building_centroids.geojson"
geojson = requests.get(url).json()

point_coords_batch = []

for f in geojson["features"]:
    geom = shape(f["geometry"])
    if geom.geom_type == "Point":
        # Wrap each point in a batch list
        point_coords_batch.append([[geom.x, geom.y]])

print(point_coords_batch[:5])  # Check first 5 points


[[[-117.60109236970347, 47.65499257572959]], [[-117.59952529804629, 47.65532576237434]], [[-117.59990985275736, 47.6551425160757]], [[-117.59953234652518, 47.6557468206869]], [[-117.59989178735704, 47.65533571100505]]]


In [55]:
point_coords_batch

[[[-117.60109236970347, 47.65499257572959]],
 [[-117.59952529804629, 47.65532576237434]],
 [[-117.59990985275736, 47.6551425160757]],
 [[-117.59953234652518, 47.6557468206869]],
 [[-117.59989178735704, 47.65533571100505]],
 [[-117.60108392059779, 47.65493215655661]],
 [[-117.60108843891226, 47.65428453173413]],
 [[-117.60108537049854, 47.65460385754599]],
 [[-117.6017152460648, 47.654290366139804]],
 [[-117.60171570950268, 47.6549491781114]],
 [[-117.60171279611392, 47.654618684108364]],
 [[-117.60171619929642, 47.65433866979389]],
 [[-117.59992357222049, 47.654956079777335]],
 [[-117.5993529577677, 47.65442330351335]],
 [[-117.6005900912993, 47.654938600412564]],
 [[-117.59983598508438, 47.65469132717483]],
 [[-117.5998874551707, 47.65442915300219]],
 [[-117.60058785890723, 47.65428095725899]],
 [[-117.6005882686511, 47.65461508757613]],
 [[-117.59869669472751, 47.6557524756083]],
 [[-117.59814618686435, 47.65575574298061]],
 [[-117.59896908519922, 47.65572562036762]],
 [[-117.5984085

In [58]:
# Flatten your current point_coords_batch
# Currently it looks like: [[[x1, y1]], [[x2, y2]], ...]
single_batch = [pt[0] for pt in point_coords_batch]

print(single_batch[:5])  # Should show: [[x1, y1], [x2, y2], ...]



[[-117.60109236970347, 47.65499257572959], [-117.59952529804629, 47.65532576237434], [-117.59990985275736, 47.6551425160757], [-117.59953234652518, 47.6557468206869], [-117.59989178735704, 47.65533571100505]]


In [64]:
sam.predict_by_points(
    point_coords_batch=single_batch,
    point_crs="EPSG:4326",
    output="mask.tif",
    dtype="uint8",
)


Some coordinates are out of the image boundary.


RuntimeError: The size of tensor a (269) must match the size of tensor b (24) at non-singleton dimension 0

In [25]:
output_masks = "building_masks.tif"

In [40]:
print(type(geojson))
print(geojson[:200] if isinstance(geojson, str) else geojson)


<class 'str'>
https://github.com/opengeos/datasets/releases/download/places/wa_building_centroids.geojson


In [41]:
from shapely.geometry import shape

# Extract point coordinates
point_coords_batch = [
    [shape(f["geometry"]).coords[0]]  # Wrap in list for batch format
    for f in geojson["features"]
    if shape(f["geometry"]).geom_type == "Point"
]


TypeError: string indices must be integers, not 'str'

In [37]:
import json
from shapely.geometry import shape

# If geojson is a string, convert to dict
if isinstance(geojson, str):
    geojson = json.loads(geojson)

features = geojson["features"]

# Extract points
points = []
for f in features:
    geom = shape(f["geometry"])
    if geom.geom_type == "Point":
        points.append([[geom.x, geom.y]])  # wrap in extra list for SAM batch

# Extract polygons for boxes
boxes = []
for f in features:
    geom = shape(f["geometry"])
    if geom.geom_type == "Polygon":
        minx, miny, maxx, maxy = geom.bounds
        boxes.append([minx, miny, maxx, maxy])

# Predict by points
if points:
    sam.predict_by_points(
        point_coords_batch=points,
        point_crs="EPSG:4326",
        output="mask_points.tif",
        dtype="uint8",
    )

# Predict by boxes
if boxes:
    sam.predict_by_boxes(
        box_coords=boxes,
        box_crs="EPSG:4326",
        output="mask_boxes.tif",
        dtype="uint8",
    )


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [53]:
import geopandas as gpd
import torch
import numpy as np
import rasterio

# Read the geojson file
gdf = gpd.read_file(geojson)

# Extract point coordinates as a list of lists
point_coords = [[point.x, point.y] for point in gdf.geometry]

# Create a list of labels (all 1s)
point_labels = [1] * len(point_coords)

# Define batch size
batch_size = 10

# Initialize an empty list to store output masks from each batch
output_masks_list = []

# Process points in batches
for i in range(0, len(point_coords), batch_size):
    batch_coords = point_coords[i : i + batch_size]
    batch_labels = point_labels[i : i + batch_size]

    # Wrap in a list to make it a batch with one element (the list of points in the current batch)
    point_coords_batch = [batch_coords]
    point_labels_batch = [batch_labels]

    # Define a temporary output file for the current batch
    temp_output_mask = f"mask_batch_{i // batch_size}.tif"

    try:
        sam.predict_by_points(
            point_coords_batch=point_coords_batch,
            point_labels_batch=point_labels_batch,
            point_crs="EPSG:4326",
            output=temp_output_mask,
            dtype="uint8",
            multimask_output=False,
        )
        output_masks_list.append(temp_output_mask)
    except Exception as e:
        print(f"Error processing batch {i // batch_size}: {e}")
        # Depending on the nature of the error, you might want to continue or break
        continue # Continue to the next batch

# Mosaic the output masks if multiple batches were processed
if len(output_masks_list) > 1:
    output_masks = "building_masks_merged.tif"
    # Use rasterio to mosaic the individual masks
    with rasterio.open(output_masks_list[0]) as src0:
        meta = src0.meta
    meta.update(compress='lzw')

    with rasterio.open(output_masks, 'w', **meta) as dst:
        for temp_mask in output_masks_list:
            with rasterio.open(temp_mask) as src:
                # Read mask data, assuming it's a single band
                mask_data = src.read(1)
                # Update the destination dataset with the new mask data where it's not nodata
                # Assuming nodata is 0
                dst.write(mask_data, 1, window=src.window(0, 0))


elif len(output_masks_list) == 1:
    # If only one batch was processed, rename the output file
    output_masks = "building_masks_merged.tif"
    import os
    os.rename(output_masks_list[0], output_masks)
else:
    print("No masks were generated.")
    output_masks = None # No output mask generated

DataSourceError: '{'type': 'FeatureCollection', 'name': 'building_centroids', 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}}, 'features': [{'type': 'Feature', 'properties': {'Class': '1', 'Confidence': 28.9054602385, 'Shape_Leng': 68.4570688978, 'Shape_Area': 292.568025508}, 'geometry': {'type': 'Point', 'coordinates': [-117.60109236970347, 47.65499257572959]}}, {'type': 'Feature', 'properties': {'Class': '1', 'Confidence': 99.4201958179, 'Shape_Leng': 97.1527845263, 'Shape_Area': 556.702899405}, 'geometry': {'type': 'Point', 'coordinates': [-117.59952529804629, 47.65532576237434]}}, {'type': 'Feature', 'properties': {'Class': '1', 'Confidence': 95.4508066177, 'Shape_Leng': 90.9169931449, 'Shape_Area': 492.940128086}, 'geometry': {'type': 'Point', 'coordinates': [-117.59990985275736, 47.6551425160757]}}, {'type': 'Feature', 'properties': {'Class': '1', 'Confidence': 91.4464533329, 'Shape_Leng': 85.6451226845, 'Shape_Area': 453.842108826}, 'geometry': {

Display the segmented masks on the map.

In [ ]:
m.add_raster(
    output_masks, cmap="jet", nodata=0, opacity=0.7, layer_name="Building masks"
)
m

![image](https://github.com/user-attachments/assets/262e1a31-1648-47d2-9e71-c85ab15b1a5c)

## Clean up the result

Remove small objects from the segmented masks, fill holes, and compute geometric properties.

In [ ]:
out_vector = "building_vector.geojson"
out_image = "buildings.tif"

In [ ]:
array, gdf = sam.region_groups(
    output_masks, min_size=200, out_vector=out_vector, out_image=out_image
)

In [ ]:
gdf.head()

![image](https://github.com/user-attachments/assets/af9ffa11-8ebe-4b42-8cba-3f5bcc4912f4)

## Regularize building footprints

Regularize the building footprints using the `regularize()` method.

In [ ]:
output_regularized = "building_regularized.geojson"
regularize(out_vector, output_regularized)

Display the regularized building footprints on the map.

In [ ]:
m = leafmap.Map()
m.add_raster(image, layer_name="Image")
style = {
    "color": "#ffff00",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0,
}
m.add_raster(out_image, cmap="tab20", opacity=0.7, nodata=0, layer_name="Buildings")
m.add_vector(
    output_regularized, style=style, layer_name="Building regularized", info_mode=None
)
m

![image](https://github.com/user-attachments/assets/b39ee029-2089-45b8-8ac0-ba0d750cec22)

## Interactive segmentation

In [ ]:
sam.show_map()

![](https://github.com/user-attachments/assets/4f487505-6e89-4892-9a70-95ab0aa69cb6)